In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

def make_dataloader(n):
    X = np.random.random((n, 2)) * 2 - 1
    Y = (X[:, 0] > X[:, 1] + 0.25).astype(int) + 1

    X = torch.tensor(X, dtype=torch.float)
    Y = torch.tensor(Y, dtype=torch.long)

    div1 = int(n*0.8)
    div2 = int(n*0.9)
    Xs = [X[:div1], X[div1:div2], X[div2:]]
    Ys = [Y[:div1], Y[div1:div2], Y[div2:]]

    print(Xs[0].shape)
    dataset = TensorDataset(Xs[0], Ys[0])
    data_loader = DataLoader(dataset, batch_size=4)
    return data_loader

In [4]:
import torch.nn as nn
from metal.mmtl.task import Task
from BERT_tasks import create_task

foo_input = nn.Linear(2, 10)
bar_input = foo_input #nn.Linear(100, 7)

foo_head = nn.Linear(10, 2)
bar_head = nn.Linear(10, 2)

foo = Task("foo_task", [make_dataloader(5000), None, None], foo_input, foo_head)
bar = Task("bar_task", [make_dataloader(2000), None, None], bar_input, bar_head)
baz = create_task('SST-2')
# tasks = [foo, bar, baz]

 19%|█▉        | 168/872 [00:00<00:00, 1676.47it/s]

torch.Size([4000, 2])
torch.Size([1600, 2])


100%|██████████| 872/872 [00:00<00:00, 1708.40it/s]


In [5]:
from metal.end_model import EndModel
from metal.mmtl.metal_model import MetalModel
from metal.mmtl.trainer import MultitaskTrainer

tasks = [baz]
model = MetalModel(tasks)


Network architecture:
MetalModel(
  (input_modules): ModuleDict(
    (SST-2): BertEncoder(
      (bert_model): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(30522, 768)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): BertLayerNorm()
          (dropout): Dropout(p=0.1)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0): BertLayer(
              (attention): BertAttention(
                (self): BertSelfAttention(
                  (query): Linear(in_features=768, out_features=768, bias=True)
                  (key): Linear(in_features=768, out_features=768, bias=True)
                  (value): Linear(in_features=768, out_features=768, bias=True)
                  (dropout): Dropout(p=0.1)
                )
                (output): BertSelfOutput(
                  (dense): Linear(in_features=768, out_features=768, bias=True)

In [ ]:
trainer = MultitaskTrainer()
trainer.train_model(model, tasks, n_epochs=1)

In [ ]:
for batch in foo.data_loaders[0]:
    X, Y = batch
    print(model(X, ['foo_task']))
    print(model.calculate_loss(X, Y, ['foo_task']))    
    print(model.calculate_output(X, ['foo_task']))    
    break